# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('IDs2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1x
  2x
  x3
  x4
  5x
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.

In [1]:
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

### Extract

In [2]:
import pandas as pd 

df = pd.read_csv('IDs2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111]


In [3]:
import requests 
import json 

def pegar_usuario(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := pegar_usuario(id)) is not None]
print(json.dumps(users, indent=2, ensure_ascii=False))

[
  {
    "id": 91,
    "name": "Danillo",
    "account": {
      "id": 98,
      "number": "00001-10",
      "agency": "9999",
      "balance": 0.0,
      "limit": 25.0
    },
    "card": {
      "id": 90,
      "number": "**** **** **** 0101",
      "limit": 10.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 92,
    "name": "Mark",
    "account": {
      "id": 99,
      "number": "00001-11",
      "agency": "9999",
      "balance": 0.0,
      "limit": 250.0
    },
    "card": {
      "id": 91,
      "number": "**** **** **** 1121",
      "limit": 100.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 93,
    "name": "Tony",
    "account": {
      "id": 100,
      "number": "00001-12",
      "agency": "9999",
      "balance": 0.0,
      "limit": 255.0
    },
    "card": {
      "id": 92,
      "number": "**** **** **** 1122",
      "limit": 950.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 94,
    "name": "Márcia",
    "account": {
      "id

### Transform

In [4]:
!pip install openai

In [5]:
# Essa parte não foi rodada pois não tenho a chave da IA-GPT
openai_api_key = ' Não tenho chave =( '

In [ ]:
import openai

openai.api_key = openai_api_key

def gerar_ai_news(user):
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-0301",
      messages=[
      {
         "role": "system", 
         "content": "Aja como um especialista em marketing bancário."
      },
      {
         "role": "user", 
         "content": f"Crie uma mensagem para {user['name']} sobre a importância de trazer seus dados para o banco para ter um limite maior(maximo de 150 caracteres)"
      }
      ]
    )
    return completion.choices[0].message.content.strip('\"')

for user in users:
    news = gerar_ai_news(user)
    print(news)
    user['news'].append({
        "description": news
    })

### Load

In [ ]:
def atualizar_user(user):
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

for user in users:
    sucess = update_user(user)
    print(f"{user['name']} atualizado? {sucess}!!")
    